In [136]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# 2020년 상반기 1~7월 도서 대출현황

In [3]:
df1 = pd.read_excel("2001.xlsx")
df1.head()

,관리번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,133200222,행정법,지은이: 김민호,박영사,2018,9791130331034,NaN,NaN,NaN,NaN,2,0,2020-01-31
1,133200221,외식 창업 컨설팅 실무,지은이: 조용범,석학당 출판사,2018,9791187621140,NaN,NaN,NaN,NaN,2,0,2020-01-31
2,133200220,(예비교사를 위한) 한국어 교육실습,박경자 지음,한국문화사,2018,9788968176098,NaN,NaN,NaN,NaN,2,0,2020-01-31
3,133200219,(예비간호사를 위한) 서비스매너,"문시정,박인아,최상호 공저",백산출판사,2018,9791188892266,NaN,NaN,NaN,NaN,2,0,2020-01-31
4,133200218,철도 경영론,김칠환 저,범한,2018,9791155961377,NaN,NaN,NaN,NaN,2,0,2020-01-31


In [4]:
df2 = pd.read_excel("2002.xlsx")
df3 = pd.read_excel("2003.xlsx")
df4 = pd.read_excel("2004.xlsx")
df5 = pd.read_excel("2005.xlsx")
df6 = pd.read_excel("2006.xlsx")
df7 = pd.read_excel("2007.xlsx")

In [5]:
df1 = df1.loc[:,['도서명','ISBN','대출건수']]
df2 = df2.loc[:,['도서명','ISBN','대출건수']]
df3 = df3.loc[:,['도서명','ISBN','대출건수']]
df4 = df4.loc[:,['도서명','ISBN','대출건수']]
df5 = df5.loc[:,['도서명','ISBN','대출건수']]
df6 = df6.loc[:,['도서명','ISBN','대출건수']]
df7 = df7.loc[:,['도서명','ISBN','대출건수']]

In [6]:
df8 = pd.concat([df1,df2,df3,df4,df5,df6,df7], axis=0)
df8.shape

(3081745, 3)

In [7]:
# ISBN기준 대출건수 집계

In [8]:
df9 = pd.pivot_table(df8, index=['도서명','ISBN'], values='대출건수', aggfunc=np.sum)

In [9]:
df9.head()

,,대출건수
도서명,ISBN,
(기도하며 함께 걷는)바울의 길,9791189299101,0
Ando,9783836509497,5
Critical literacy pedagogy for bilingual preservice teachers,9789811079344,0
Dangerous Days of Daniel X,9780316119702,0
"Grammar Book: An ESL/EFL Teachers Course,",9780838447253,0


In [10]:
df9.shape

(396379, 1)

In [11]:
df9['대출건수'].max()

31311

In [12]:
df9[df9['대출건수']>30000]

,,대출건수
도서명,ISBN,
(대하역사만화) 박시백의 조선왕조실록,9788958620396,31311


In [13]:
# 세종 국립도서관 대출건수 상위 100 도서 선정

In [14]:
df10 = df9.sort_values(by=['대출건수'], ascending=False)
df10 = df10.iloc[0:100,:]

In [15]:
df10

,,대출건수
도서명,ISBN,
(대하역사만화) 박시백의 조선왕조실록,9788958620396,31311
토지,9788960532410,22044
삼국지,9788955708738,20335
(대하역사만화) 박시백의 조선왕조실록,9788958620358,17539
크레파스 미술동화,9788965092759,15806
...,...,...
Why? 동물,9788930202367,3871
Why? 영웅이야기,9788930232449,3871
미생,9788960865815,3849


In [16]:
df10.to_csv('sejong100.csv')

In [17]:
df10.reset_index(inplace=True)
df10

,도서명,ISBN,대출건수
0,(대하역사만화) 박시백의 조선왕조실록,9788958620396,31311
1,토지,9788960532410,22044
2,삼국지,9788955708738,20335
3,(대하역사만화) 박시백의 조선왕조실록,9788958620358,17539
4,크레파스 미술동화,9788965092759,15806
...,...,...,...
95,Why? 동물,9788930202367,3871
96,Why? 영웅이야기,9788930232449,3871
97,미생,9788960865815,3849
98,"총, 균, 쇠",9788970128856,3844


In [18]:
df10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   도서명     100 non-null    object
 1   ISBN    100 non-null    object
 2   대출건수    100 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [19]:
isbn = df10['ISBN'].astype(str)

In [20]:
isbn_list = list(isbn)

In [21]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import urljoin
import requests
import xmltodict
import time

In [22]:
Data = []
for i in isbn_list:
    # print(i)
    url='http://data4library.kr/api/srchDtlList?authKey=99e9007714996192cfe6b56a179c274e21d029008c4813ab1c3fff8dca8ef4a1&isbn13='+i+'&loaninfoYN=Y&displayInfo=age'
    req = requests.get(url).content
    xmlObject = xmltodict.parse(req)
    allData = xmlObject['response']['detail']['book']
    Data.append(allData)

In [23]:
Data

[OrderedDict([('no', '1'),
              ('bookname', '(대하역사만화)박시백의 조선왕조실록'),
              ('publication_date', None),
              ('authors', '박시백 글·그림'),
              ('publisher', '휴머니스트'),
              ('class_no', '911.05'),
              ('publication_year', None),
              ('bookImageURL',
               'https://bookthumb-phinf.pstatic.net/cover/015/361/01536143.jpg?type=m1&udate=20150716'),
              ('isbn', '8958620390'),
              ('isbn13', '9788958620396'),
              ('description',
               '거침없는 야심가인 수양대군과 출세주의자 한명회가 계유정난을 통해 홀로 남겨진 소년 임금 단종을 몰아낸다. 조카인 단종을 몰아내고 왕이 된 세조는 왕권을 강화하여 민생안정과 부국강병에 힘을 쓰며 중앙집권 시스템을 강화하려 하는데, 왕의 비호 아래 부와 힘을 축적한 공신들의 부패는 끊이질 않는다.')]),
 OrderedDict([('no', '1'),
              ('bookname', '토지 :박경리 대하소설'),
              ('publication_date', '2012'),
              ('authors', '지은이: 박경리'),
              ('publisher', '마로니에북스'),
              ('class_no', '813.6'),
              ('publication_year', '2012'),
              ('

In [25]:
type(Data)

list

In [27]:
Detail = pd.DataFrame(Data)

In [28]:
Detail.to_csv("detailInfo.csv")